In [ ]:
import pandas as pd
import warnings
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
warnings.simplefilter(action='ignore')

In [ ]:
# 1. 페이지 접근
# 2. 주소 검색 탭 선택
# 3. 검색어(구) 입력
# 4. 검색 버튼 입력
# 5. 검색 결과 리스트 가져오기(뷰티풀숲)
# 6. 데이터 수집(매장 이름, 주소)
# 7. 전체 데이터 수집

In [ ]:
# 1. 이디야 페이지 접근
url = 'https://www.ediya.com/contents/find_store.html#c'
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
# 2. 주소 검색 탭 선택
#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a
driver.find_element(By.CSS_SELECTOR, '.store_search_pop li:nth-child(2) > a').click()

In [ ]:
# 3. 검색어(구) 입력

# 스타벅스 매장 구 이름 가져오기
starbucks_df = pd.read_csv('starbucks.csv', index_col=0)
gu_list = list(starbucks_df['gu'].unique())
gu_list = [str('서울 ') + gu for gu in gu_list]
len(gu_list), gu_list[:3] # 강서구 중구

(25, ['서울 강남구', '서울 강북구', '서울 강서구'])

In [ ]:
# 구별 검색어 입력
search_keyword = driver.find_element(By.CSS_SELECTOR, '#keyword')
search_keyword.clear()
search_keyword.send_keys(gu_list[1])

In [ ]:
# 구별 반복문 테스트
for gu in gu_list:
    search_keyword.send_keys(gu)
    search_keyword.clear()

In [ ]:
# 4. 검색 버튼 입력
search_button = driver.find_element(By.CSS_SELECTOR, '#keyword_div button')
search_button.click()

In [ ]:
# 5. 검색 결과 리스트 가져오기(beautifulsoup 파싱)
html = driver.page_source
dom = BeautifulSoup(html, 'html.parser')
contents = dom.select('#placesList li')
len(contents)

20

In [ ]:
# 6. 매장이름, 주소 데이터 수집
title = contents[0].select_one('dt').text
address = contents[0].select_one('dd').text
gu_name = address.split()[1]
title, address, gu_name

('4.19사거리점', '서울 강북구 삼양로 501 (수유동)', '강북구')

In [ ]:
driver.close()

In [ ]:
# 7. 전체 데이터 수집
import pandas as pd

url = 'https://www.ediya.com/contents/find_store.html#c'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

driver.find_element(By.CSS_SELECTOR, '.store_search_pop li:nth-child(2) > a').click()
search_keyword = driver.find_element(By.CSS_SELECTOR, '#keyword')
search_button = driver.find_element(By.CSS_SELECTOR, '#keyword_div button')
time.sleep(3)

# 5. 전체 데이터 수집
datas = []

for gu in gu_list:
    try:
        # (1) 검색어 창 초기화
        search_keyword.clear()
        time.sleep(2)
        # (2) 검색어 입력
        search_keyword.send_keys(gu)
        # (3) 검색 버튼 클릭
        search_button.click()
        time.sleep(1)
        # (4) beautifulsoup 파싱
        html = driver.page_source
        dom = BeautifulSoup(html, 'html.parser')
        contents = dom.select('#placesList li')
        # (5) 전체 데이터 수집

        for content in contents:
            title = content.select_one('dt').text
            address = content.select_one('dd').text
            gu_name = address.split()[1]

            datas.append({
                'title': title,
                'address': address,
                'gu': gu_name,
            })
    except Exception as e:
        # 에러가 발생할 경우 에러메시지 출력 후 드라이버 종료
        print(e)
        driver.close()

driver.close()
df = pd.DataFrame(datas)
df.tail()

,title,address,gu
652,중랑묵동점,"서울 중랑구 동일로 932 (묵동, 묵동자이아파트)",중랑구
653,중랑역점,서울 중랑구 망우로 198 (상봉동),중랑구
654,중화동점,서울 중랑구 동일로129길 1 (중화동),중랑구
655,중화역점,"서울 중랑구 동일로 815, 1층",중랑구
656,화랑대역점,"서울 중랑구 신내로25가길 2 (묵동, 현동학당)",중랑구


In [ ]:
# 8. 위도 경도 컬럼 추가
import googlemaps

gmaps_key = ""
gmaps = googlemaps.Client(key=gmaps_key)

In [ ]:
import numpy as np

df["lat"] = np.nan
df["lng"] = np.nan

In [ ]:
from tqdm import tqdm_notebook

for idx, rows in tqdm_notebook(df.iterrows()):
    try:
        address = rows["address"]
        tmp = gmaps.geocode(address, language="ko")
        tmp[0].get("formatted_address")

        lat = tmp[0].get("geometry")["location"]["lat"]
        lng = tmp[0].get("geometry")["location"]["lng"]

        df.loc[idx, "lat"] = lat
        df.loc[idx, "lng"] = lng
    except:
        continue

0it [00:00, ?it/s]

In [ ]:
# 9. 전체 주소에 대한 값을 불러오지 못할 경우, null(NaN) 값 대처
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   title    657 non-null    object 
 1   address  657 non-null    object 
 2   gu       657 non-null    object 
 3   lat      648 non-null    float64
 4   lng      648 non-null    float64
dtypes: float64(2), object(3)
memory usage: 25.8+ KB


In [ ]:
# 주소가 검색되지 않아 lat, lng 컬럼이 NaN인 경우
df[df['lat'].isnull() | df['lng'].isnull()]

,title,address,gu,lat,lng
21,신사점,서울 강남구 도산대로 118 (논현동),강남구,NaN,NaN
193,쌍문동점,서울 도봉구 노해로 162 (쌍문동),도봉구,NaN,NaN
245,신촌하나로마트점,"서울 마포구 신촌로 66 (노고산동, 농협중앙회)",마포구,NaN,NaN
256,가재울점,"서울 서대문구 수색로 100 (북가좌동, DMC래미안e편한세상)",서대문구,NaN,NaN
319,월곡중앙점,"서울 성북구 종암로 167 (하월곡동, 동일하이빌뉴시티)",성북구,NaN,NaN
333,가든파이브테크노관점,"서울 송파구 충민로 66 (문정동, 가든파이브라이프)",송파구,NaN,NaN
366,잠실역점,"서울 송파구 송파대로 567 (잠실동, 잠실주공아파트)",송파구,NaN,NaN
404,라이프점,"서울 영등포구 63로 40 (여의도동, 라이프오피스텔)",영등포구,NaN,NaN
449,연신내중앙점,서울 은평구 통일로 825 (대조동),은평구,NaN,NaN


In [ ]:
# NaN값인 주소 중, 검색이 되지 않는 주소가 있을 수도 있습니다.
# 생긴지 얼마 되지 않아 업데이트가 안됐거나 등 사유가 있을 수 있습니다.
tmp = gmaps.geocode('서울 마포구 신촌로 66 (노고산동, 농협중앙회)', language='ko')
tmp

[]

In [ ]:
# 해결 예시 1
# 수작업으로 찾아서 채워주는 방법
# bing 에서 해당 주소 검색 후, 위도 경도 값 확인하기
# https://www.bing.com/maps/?cp=37.485228%7E126.877413&lvl=16.0
df.loc[457, ['lat']] = 37.5195583
df.loc[457, ['lng']] = 126.9391694
df.loc[457]

title                                라이프점
address    서울 영등포구 63로 40 (여의도동, 라이프오피스텔)
gu                                   영등포구
lat                             37.519558
lng                            126.939169
Name: 457, dtype: object

In [ ]:
# 해결 예시 2
# null(NaN) 값이 있는 행 제외
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# 본 강의에서는 null(NaN) 값 제외로 진행합니다.
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 648 entries, 0 to 656
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   title    648 non-null    object 
 1   address  648 non-null    object 
 2   gu       648 non-null    object 
 3   lat      648 non-null    float64
 4   lng      648 non-null    float64
dtypes: float64(2), object(3)
memory usage: 30.4+ KB


In [ ]:
# csv 파일 저장
df.to_csv('./ediya.csv', encoding='utf-8')

In [ ]:
# 파일 읽기
ediay_df = pd.read_csv('./ediya.csv', index_col=0, encoding='utf-8')
ediay_df

,title,address,gu,lat,lng
0,강남YMCA점,서울 강남구 논현동,강남구,37.513679,127.031712
1,강남구청역아이티웨딩점,"서울 강남구 학동로 338 (논현동, 강남파라곤)",강남구,37.516551,127.040139
2,강남논현학동점,서울 강남구 논현로131길 28 (논현동),강남구,37.515190,127.027554
3,강남대치점,"서울 강남구 역삼로 415 (대치동, 성진빌딩)",강남구,37.501434,127.052328
4,강남도산점,서울 강남구 도산대로37길 20 (신사동),강남구,37.522282,127.031480
...,...,...,...,...,...
652,중랑묵동점,"서울 중랑구 동일로 932 (묵동, 묵동자이아파트)",중랑구,37.613779,127.077524
653,중랑역점,서울 중랑구 망우로 198 (상봉동),중랑구,37.593285,127.074889
654,중화동점,서울 중랑구 동일로129길 1 (중화동),중랑구,37.601957,127.086627
655,중화역점,"서울 중랑구 동일로 815, 1층",중랑구,37.603129,127.078889
